# Web Scraping Job Vacancies

### Introduction

En este proyecto, crearemos un raspador web para extraer listados de empleos de una plataforma de búsqueda de empleo popular. Extraeremos títulos de puestos, empresas, ubicaciones, descripciones de puestos y otra información relevante. Estos son los pasos principales que seguiremos en este proyecto:

1. Configurar nuestro entorno de desarrollo.
2. Comprender los conceptos básicos del web scraping.
3. Analizar la estructura del sitio web de nuestra plataforma de búsqueda de empleo.
4. Escribir el código Python para extraer datos laborales de nuestra plataforma de búsqueda de empleo.
    2. Guardar los datos en un archivo CSV.
    3. Probar nuestro web scraper y refinar nuestro código según sea necesario.

#### Requisitos previos

Antes de comenzar este proyecto, debe tener algunos conocimientos básicos de programación en Python y estructura HTML. Además, es posible que desee utilizar los siguientes paquetes en su entorno Python:

- `requests`
- `BeautifulSoup`
- `csv`
- `datetime`


### Paso 1: Configurar nuestro entorno de desarrollo.



In [4]:
!pip install pandas matplotlib seaborn requests selenium

### Paso 2: Comprender los conceptos básicos del web scraping.

#### Conceptos Básicos de Web Scraping

Aquí tienes los conceptos básicos de **web scraping** de manera general:

1. **HTTP Requests**: Web scraping comienza con el envío de solicitudes HTTP (GET o POST) al servidor web para obtener el contenido de una página. Las bibliotecas de Python como `requests` permiten hacer solicitudes y obtener el HTML de la página.

2. **Parsing HTML**: Una vez que se obtiene el HTML, es necesario analizarlo (parsing) para extraer los datos específicos. Esto implica navegar y seleccionar elementos HTML como etiquetas, clases y atributos, utilizando bibliotecas como **BeautifulSoup** o **lxml**.

3. **XPath y Selectores CSS**: XPath y selectores CSS son métodos para identificar elementos específicos en un documento HTML. XPath es útil para seleccionar elementos a través de rutas, mientras que los selectores CSS permiten especificar etiquetas y clases directamente, por ejemplo, `.class-name` o `#id`.

4. **Navegación en el DOM**: El DOM (Document Object Model) es la estructura jerárquica del HTML. Comprender la estructura del DOM es clave para localizar y extraer datos con precisión, ya que los datos se encuentran en nodos que se pueden recorrer de arriba hacia abajo o de izquierda a derecha.

5. **Manejo de Datos Dinámicos**: Algunos sitios generan contenido dinámicamente con JavaScript, lo que significa que los datos no están disponibles en el HTML inicial. Para ello, herramientas como **Selenium** o **Playwright** emulan la interacción con el navegador y permiten obtener el HTML final, incluido el contenido dinámico.

6. **Exportación y Almacenamiento de Datos**: Los datos extraídos suelen almacenarse en formatos estructurados como CSV, JSON o bases de datos SQL/NoSQL, lo que permite manipularlos y analizarlos fácilmente.

7. **Respeto por las Políticas de Scraping (Robots.txt)**: Es fundamental respetar las reglas de `robots.txt` de cada sitio web, que especifican las páginas y los límites de frecuencia permitidos para los bots, y tener en cuenta el buen uso de las tasas de solicitud para evitar sobrecargar el servidor.

Estos conceptos forman la base del web scraping y ayudan a estructurar un proyecto de scraping de manera eficiente y respetuosa con las políticas del sitio web.


### Paso 3: Analizar la estructura del sitio web de nuestra plataforma de búsqueda de empleo.



En este caso, se ha seleccionado el buscador de empleo **Computrabajo** en el país de Colombia.

**Enlace**: [https://co.computrabajo.com](https://co.computrabajo.com)

Para el análisis HTML, es importante comprender cómo funciona el buscador y cómo organiza y categoriza las vacantes en su página web. Esto se realiza a través del estudio de la estructura HTML, identificando los `<div>` correspondientes a los elementos deseados, que en este caso son las cajas que muestran las vacantes de empleo.

#### Consideraciones

**Para mejorar:** Es posible crear un archivo para cada vacante que se actualice. Actualmente, solo se cuenta con las primeras 20 vacantes, es decir, de la primera página. Se podría optimizar el código para que busque en todo el número de vacantes y colocar la fecha de la consulta (código: implementar la navegación a las demás páginas).

Además, sería útil incluir las descripciones de cada empleo, ya que a menudo contienen información importante. Para lograr esto, se debe acceder a cada vacante, lo que implica cambiar la URL. También sería recomendable crear una base de datos para las vacantes seleccionadas, ya que no todas están disponibles en la plataforma de búsqueda. Esto se podría lograr extrayendo los nombres de los puestos de empleo directamente del buscador.


### Paso 3: Escribir el código Python para extraer datos laborales de nuestra plataforma de búsqueda de empleo.

#### 3.1. Solicitud de empleo y creación de URL

In [40]:
# Función para solicitar el trabajo al usuario
def enter_job_url():
    job = input('Ingresa el trabajo que quieres buscar: ').strip()
    listjob = job.split()
    url = 'https://co.computrabajo.com/' + 'trabajo-de-' + "-".join(listjob)
    print(f"URL a buscar: {url}")
    return url


#### 3.2. Solicitud HTML al sitio: Extracción de datos

In [42]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd

def analysis_HTML():
    # Obtén la URL para buscar el trabajo
    url = enter_job_url()

    # Configurar Selenium y abrir la página web
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service)

    driver.get(url)

    # Esperar explícitamente hasta que los elementos carguen (máximo 20 segundos)
    try:
        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'box_offer'))
        )
    except:
        print("No se pudo cargar la página a tiempo.")
        driver.quit()
        return

    # Extraer ofertas
    def extract_offers():
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        # Extraer el número total de ofertas desde el h1 dentro del div box_title
        num_offers_element = soup.select_one('div.box_title h1.title_page span.fwB')

        if num_offers_element:
            # Obtener el texto dentro del span y procesarlo
            num_offers_text = num_offers_element.text.strip()
            # print(f"Texto extraído del span: '{num_offers_text}'")  # Imprimir el texto para depuración
    
            try:
                # Intentar convertir a número entero
                total_offers = int(num_offers_text.replace('.', ''))
                print(f"Total de ofertas: {total_offers}")
            except ValueError:
                print(f"Error al convertir a entero: '{num_offers_text}' no es un número válido.")
        else:
            print("No se encontró el número de ofertas.")

        # Usar una lista para almacenar la información de las ofertas
        job_list = []

        # Iterar sobre los primeros 20 artículos
        for i in range(20):
            offer_tag = soup.find('article', {'data-lc': f'ListOffers-Score4-{i}'})
            
            if not offer_tag:
                continue  # Si no encuentra la oferta, pasa a la siguiente

            # Extraer título, enlace, empresa, ubicación, salario y tiempo de publicación
            title_tag = offer_tag.find('h2', class_='fs18 fwB')
            title = title_tag.get_text(strip=True) if title_tag else 'No disponible'

            link_tag = offer_tag.find('a', class_='js-o-link fc_base')
            link = link_tag['href'] if link_tag else 'No disponible'

            company_tag = offer_tag.find('a', class_='fc_base t_ellipsis')
            company = company_tag.get_text(strip=True) if company_tag else 'No disponible'

            location_tag = offer_tag.find('p', class_='fs16 fc_base mt5')
            location = location_tag.get_text(strip=True) if location_tag else 'No disponible'

            salary_tag = offer_tag.find('div', class_='fs13 mt15')
            salary = salary_tag.get_text(strip=True) if salary_tag else 'No especificado'

            time_posted_tag = offer_tag.find('p', class_='fs13 fc_aux mt15')
            time_posted = time_posted_tag.get_text(strip=True) if time_posted_tag else 'No disponible'

            # Agregar los datos a la lista de ofertas
            job_list.append({
                'title': title,
                'link': link,
                'company': company,
                'location': location,
                'salary': salary,
                'time_posted': time_posted
            })

        return job_list

    # B. Guardar datos en un archivo CSV
    def save_csv():
        offers = extract_offers()
        if offers:
            df = pd.DataFrame(offers)
            df.to_csv('Job_Offers.csv', index=False)
            print("Datos guardados en Job_Offers.csv")
        else:
            print("No se encontraron ofertas para guardar.")
    
    save_csv()
    driver.quit()

analysis_HTML()
# C. Probar el programa

Ingresa el trabajo que quieres buscar:  analista de datos


URL a buscar: https://co.computrabajo.com/trabajo-de-analista-de-datos
Total de ofertas: 1391
Datos guardados en Job_Offers.csv


### 3. Análisis de las vacantes (a vuestro criterio)

In [ ]:
# Leer el CSV 
# Qué quiero analizar encontrar un (vs)  por ejemplo ¿qué compañia paga más?, ¿que tan cerca queda de mi hogar?
